In [2]:
import os
from ames.harvesters import file_mapping, get_usage, build_usage
from ames.harvesters import build_aggregate, aggregate_usage
from ames.harvesters import get_caltechdata, get_history
from ames.matchers import add_usage, submit_report

if os.path.isdir("data") == False:
    os.mkdir("data")
os.chdir("data")

production = True
collection = "caltechdata.ds"

get_caltechdata(collection, production)
mapping = file_mapping(collection)



  0% (34 of 8729) |                      | Elapsed Time: 0:00:00 ETA:   0:00:25IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

100% (8729 of 8729) |####################| Elapsed Time: 0:02:05 Time:  0:02:05


In [25]:
import requests

downloads = {}

days = 1
while days < 31:
    print(days)
    response = requests.get(f'https://stats.tind.io/index.php?date=2021-12-{days:02}&expanded=1&filter_limit=-1&format=JSON&idSite=1161&method=Actions.getDownloads&module=API&period=day&segment=&token_auth=TOKEN')
    for dl in response.json()[0]['subtable']:
        unique = dl['nb_uniq_visitors']
        idv = 'https://data.caltech.edu'+ dl['label']
        if idv in mapping:
            label = mapping[idv]
            if label in downloads:
                downloads[label] = downloads[label] + unique
            else:
                downloads[label] = unique
    days = days + 1
        
for d in downloads:
    print(d)
    print(downloads[d])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
293
37
1173
8
1490
1
1655
26
1996
7
1882
4
1455
1
1493
1
1290
3
1551
1
1249
25
1813
1
1692
1
1991
36
1892
13
1947
4
2012
23
250
2
306
1
957
1
210
3
1092
14
1446
1
249
7
2185
1
1819
1
2177
1
1296
3
1827
1
1382
2
1826
1
1220
2
1572
1
1955
2
295
2
1444
7
2202
1
1495
1
247
3
1482
1
1981
1
254
1
1632
5
1256
3
270
3
1226
2
2065
1
1178
1
1538
1
1177
4
1703
6
1574
1
756
1
437
1
956
1
1311
1
1229
1
1802
1
278
2
1876
1
1650
1
1803
1
1690
1
1989
5
1693
6
1103
6
1668
2
1958
2
1067
1
1895
3
279
1
1472
1
1898
1
1327
3
1945
1
2156
4
1798
1
1812
1
248
3
251
2
2075
1
2147
1
269
1
2071
1
2070
1
2207
2
2067
1
2068
1
1407
13
1701
4
1270
1
2178
1
372
1
2086
1
1970
2
1651
1
1317
12
2061
1
2060
1
2062
1
2074
1
1287
1
1264
15
1160
1
1237
2
1571
1
228
1
2011
6
291
2
923
1
1293
3
2113
1
2150
8
1956
1
1670
2
1443
2
1115
1
1114
1
2126
2
1792
1
1189
1
547
1
1678
1
2102
1


In [33]:
from ames.harvesters import get_records
from py_dataset import dataset

dot_paths = ['._Key','.publicationYear','.electronic_location_and_access']
keys = dataset.keys(collection)
all_metadata = get_records(dot_paths,'dois',collection,keys)

rec_by_pub_year = {}
dl_by_pub_year = {}

for record in all_metadata:
    key = record['_Key']
    year = record['publicationYear']
    if 'electronic_location_and_access' in record:
        if year in rec_by_pub_year:
            rec_by_pub_year[year] += 1
        else:
            rec_by_pub_year[year] = 1
    if key in downloads:
        if year in dl_by_pub_year:
            dl_by_pub_year[year] += 1
        else:
            dl_by_pub_year[year] = 1

print(rec_by_pub_year['2017'])
print(rec_by_pub_year['2018'])
print(rec_by_pub_year['2019'])
print(rec_by_pub_year['2020'])
print(rec_by_pub_year['2021'])
print(dl_by_pub_year['2017']/rec_by_pub_year['2017'])
print(dl_by_pub_year['2018']/rec_by_pub_year['2018'])
print(dl_by_pub_year['2019']/rec_by_pub_year['2019'])
print(dl_by_pub_year['2020']/rec_by_pub_year['2020'])
print(dl_by_pub_year['2021']/rec_by_pub_year['2021'])        

49
134
149
420
367
0.10204081632653061
0.05223880597014925
0.12751677852348994
0.0880952380952381
0.10626702997275204
